In [1]:
import pandas as pd, numpy as np, matplotlib.pyplot as plt

In [2]:
from sklearn.cluster import KMeans

In [3]:
df = pd.read_csv("stanfordMOOCForumPostsSet.csv")

In [4]:
df.head()

,Text,Opinion(1/0),Question(1/0),Answer(1/0),Sentiment(1-7),Confusion(1-7),Urgency(1-7),CourseType,forum_post_id,course_display_name,forum_uid,created_at,post_type,anonymous,anonymous_to_peers,up_count,comment_thread_id,reads
0,Interesting! How often we say those things to ...,1,0,0,6.5,2.0,1.5,Education,5225177f2c501f0a00000015,Education/EDUC115N/How_to_Learn_Math,30CADB93E6DE4711193D7BD05F2AE95C,02-09-2013 22:55,Comment,False,False,0.0,5221a8262cfae31200000001,41.0
1,"What is \Algebra as a Math Game\"" or are you j...",0,1,0,4.0,5.0,3.5,Education,5207d0e9935dfc0e0000005e,Education/EDUC115N/How_to_Learn_Math,37D8FAEE7D0B94B6CFC57D98FD3D0BA5,11-08-2013 17:59,Comment,False,False,0.0,520663839df35b0a00000043,55.0
2,I like the idea of my kids principal who says ...,1,0,0,5.5,3.0,2.5,Education,52052c82d01fec0a00000071,Education/EDUC115N/How_to_Learn_Math,CC11480215042B3EB6E5905EAB13B733,09-08-2013 17:53,Comment,False,False,0.0,51e59415e339d716000001a6,25.0
3,"From their responses, it seems the students re...",1,0,0,6.0,3.0,2.5,Education,5240a45e067ebf1200000008,Education/EDUC115N/How_to_Learn_Math,C717F838D10E8256D7C88B33C43623F1,23-09-2013 20:28,CommentThread,False,False,0.0,None,0.0
4,"The boys loved math, because \there is freedom...",1,0,0,7.0,2.0,3.0,Education,5212c5e2dd10251500000062,Education/EDUC115N/How_to_Learn_Math,F83887D68EA48964687C6441782CDD0E,20-08-2013 01:26,CommentThread,False,False,0.0,None,3.0


In [5]:
df['course_display_name'].unique()

array(['Education/EDUC115N/How_to_Learn_Math',
       'HumanitiesScience/Stats216/Winter2014',
       'HumanitiesSciences/EP101/Environmental_Physiology',
       'HumanitiesScience/StatLearning/Winter2014',
       'GlobalHealth/WomensHealth/Winter2014',
       'HumanitiesSciences/Econ1V/Summer2014',
       'HumanitiesSciences/Econ-1/Summer2014', nan,
       'Medicine/HRP258/Statistics_in_Medicine',
       'Medicine/SciWrite/Fall2013',
       'Medicine/SURG210/Managing_Emergencies_What_Every_Doctor_Must_Know',
       'Medicine/MedStats/Summer2014'], dtype=object)

In [6]:
df['CourseType'].unique()

array(['Education', nan, 'Humanities', 'Medicine'], dtype=object)

In [7]:
len(df[df['Question(1/0)']==1][df['Opinion(1/0)']==1])

C:\Users\HP\AppData\Local\Temp\ipykernel_22740\2890159968.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  len(df[df['Question(1/0)']==1][df['Opinion(1/0)']==1])


1325

In [8]:
partset = df[df['Question(1/0)']==1][['Urgency(1-7)','up_count','reads','Confusion(1-7)','Opinion(1/0)']]

In [9]:
partset.describe()

,Urgency(1-7),up_count,reads,Confusion(1-7),Opinion(1/0)
count,5959.000000,5957.000000,5957.000000,5959.000000,5959.000000
mean,4.602786,0.506463,369.893906,4.526850,0.222353
std,1.169059,1.716520,980.799964,0.656481,0.415862
min,1.000000,0.000000,0.000000,2.000000,0.000000
25%,4.000000,0.000000,47.000000,4.000000,0.000000
50%,5.000000,0.000000,94.000000,4.500000,0.000000
75%,5.500000,0.000000,232.000000,5.000000,0.000000
max,7.000000,46.000000,9452.000000,7.000000,1.000000


In [10]:
temp = partset

In [11]:
temp['up_count']/=max(partset['up_count'])

In [12]:
temp['reads']/=max(partset['reads'])

In [13]:
temp['Confusion(1-7)']/=max(partset['Confusion(1-7)'])
temp['Urgency(1-7)']/=max(partset['Urgency(1-7)'])

In [14]:
temp.describe()

,Urgency(1-7),up_count,reads,Confusion(1-7),Opinion(1/0)
count,5959.000000,5957.000000,5957.000000,5959.000000,5959.000000
mean,0.657541,0.011010,0.039134,0.646693,0.222353
std,0.167008,0.037316,0.103766,0.093783,0.415862
min,0.142857,0.000000,0.000000,0.285714,0.000000
25%,0.571429,0.000000,0.004972,0.571429,0.000000
50%,0.714286,0.000000,0.009945,0.642857,0.000000
75%,0.785714,0.000000,0.024545,0.714286,0.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000


In [15]:
Y = temp['Urgency(1-7)']

In [16]:
X = temp[['up_count','reads','Confusion(1-7)','Opinion(1/0)']]

In [17]:
from sklearn.decomposition import PCA

In [18]:
X = X.fillna(0)

In [19]:
X.isnull().sum()

up_count          0
reads             0
Confusion(1-7)    0
Opinion(1/0)      0
dtype: int64

In [41]:
X_reduced = PCA().fit_transform(X)

In [90]:
X_reduced

array([[-314.76982896,   -0.45312356,   -0.48113023],
       [-349.76982846,   -0.447281  ,   -0.48194334],
       [-352.76981669,   -0.44612032,   -0.9820126 ],
       ...,
       [-319.76970889,    0.54507064,    1.52007165],
       [-225.76983022,   -0.46798033,   -0.47906263],
       [-166.76983106,   -0.47782921,   -0.47769196]])

In [16]:
km = KMeans(n_clusters=13,n_init=10,max_iter=300,tol=1e-04,random_state=0)

In [23]:
from sklearn.model_selection import train_test_split

In [24]:
X_train, x_test, Y_train, y_test = train_test_split(X,Y)


In [25]:
print(X_train.describe())
X_train = X_train.values
x_test = x_test.values
Y_train = Y_train.values
y_test = y_test.values

          up_count        reads  Confusion(1-7)  Opinion(1/0)
count  4469.000000  4469.000000     4469.000000   4469.000000
mean      0.011105     0.040220        0.647173      0.222197
std       0.039550     0.107548        0.094628      0.415770
min       0.000000     0.000000        0.357143      0.000000
25%       0.000000     0.004972        0.571429      0.000000
50%       0.000000     0.010051        0.642857      0.000000
75%       0.000000     0.025074        0.714286      0.000000
max       1.000000     1.000000        1.000000      1.000000


In [103]:
from sklearn.linear_model import LinearRegression

In [104]:
model = LinearRegression()

In [111]:
model.fit(X_train,Y_train)

LinearRegression()

In [112]:
y_pred = model.predict(x_test)

In [113]:
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
score=r2_score(y_test,y_pred)
print('r2 socre is ',score)
print('mean_sqrd_error is==',mean_squared_error(y_test,y_pred))
print('root_mean_squared error of is==',np.sqrt(mean_squared_error(y_test,y_pred)))

r2 socre is  0.2366995779427632
mean_sqrd_error is== 0.021121903665015537
root_mean_squared error of is== 0.14533376643098306


In [59]:
km.fit(X_reduced)

KMeans(n_clusters=13, random_state=0)

In [65]:
Y

1        3.5
22       3.5
33       4.5
46       2.5
50       5.0
        ... 
29593    5.5
29596    3.0
29598    1.0
29601    5.5
29602    5.5
Name: Urgency(1-7), Length: 5959, dtype: float64

In [62]:
km.predict(X_reduced)

array([0, 0, 0, ..., 0, 8, 8])

In [63]:
Y

1        3.5
22       3.5
33       4.5
46       2.5
50       5.0
        ... 
29593    5.5
29596    3.0
29598    1.0
29601    5.5
29602    5.5
Name: Urgency(1-7), Length: 5959, dtype: float64

In [66]:
from sklearn.cluster import DBSCAN

In [67]:
dbsc = DBSCAN()

In [69]:
dbsc.fit(X_reduced)

DBSCAN()

In [71]:
res = dbsc.fit_predict(X_reduced)

In [72]:
res

array([ 0,  1, -1, ..., -1, -1, -1], dtype=int64)

In [26]:
#neural network
import torch
import torch.nn as nn
import torch.utils.data as td

In [27]:
class ShauryaNet(nn.Module):
    def __init__(self):
        super(ShauryaNet, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(4, 100),
            nn.ReLU(),
            nn.Linear(100, 100),
            nn.ReLU(),
            nn.Linear(100, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        x = self.net(x)
        return x

In [28]:
net = ShauryaNet()
inp = torch.randn(10, 4)
print(net(inp))

tensor([[0.4881],
        [0.4708],
        [0.5176],
        [0.4881],
        [0.4885],
        [0.4931],
        [0.5065],
        [0.4867],
        [0.4747],
        [0.4798]], grad_fn=<SigmoidBackward0>)


In [29]:
print(X_train.shape)
print(X_train[:5])

(4469, 4)
[[4.34782609e-01 1.14578925e-01 5.71428571e-01 0.00000000e+00]
 [0.00000000e+00 6.34786289e-04 5.71428571e-01 1.00000000e+00]
 [0.00000000e+00 1.37537029e-03 6.42857143e-01 1.00000000e+00]
 [0.00000000e+00 6.77105375e-03 5.71428571e-01 0.00000000e+00]
 [0.00000000e+00 1.12145578e-02 5.71428571e-01 0.00000000e+00]]


In [30]:
# Create a dataset and loader for the training data and labels
train_x = torch.Tensor(X_train).float()
train_y = torch.Tensor(Y_train).float()
train_ds = td.TensorDataset(train_x, train_y)
print(train_x[:5])
train_loader = td.DataLoader(train_ds, batch_size=20,
    shuffle=False, num_workers=1)

# Create a dataset and loader for the test data and labels
test_x = torch.Tensor(x_test).float()
test_y = torch.Tensor(y_test).float()
test_ds = td.TensorDataset(test_x,test_y)
test_loader = td.DataLoader(test_ds, batch_size=20,
    shuffle=False, num_workers=1)
print('Ready to load data')

tensor([[4.3478e-01, 1.1458e-01, 5.7143e-01, 0.0000e+00],
        [0.0000e+00, 6.3479e-04, 5.7143e-01, 1.0000e+00],
        [0.0000e+00, 1.3754e-03, 6.4286e-01, 1.0000e+00],
        [0.0000e+00, 6.7711e-03, 5.7143e-01, 0.0000e+00],
        [0.0000e+00, 1.1215e-02, 5.7143e-01, 0.0000e+00]])
Ready to load data


In [31]:
def train(model, data_loader, optimizer, loss_criteria):
    # Set the model to training mode
    correct = 0
    model.train()
    train_loss = 0
    batch_size = len(data_loader.dataset)
#     print(next(iter(train_loader)))
    for batch, tensor in enumerate(data_loader):
        data, target = tensor
        #feedforward
        optimizer.zero_grad()
        out = model(data)
#         print(data, out, target)
#         print(out.shape, target.shape)
        loss = loss_criteria(out, target)
        train_loss += loss.item()
#         print(loss.item())
        # backpropagate
        loss.backward()
        optimizer.step()
        correct += torch.sum(abs((out * 13) - (target * 13)) >= 0.5).item()

    train_acc = 100. * correct / (batch_size + 1)
    #Return average loss
    avg_loss = train_loss / (batch+1)
#     print(f'Training set: {train_loss:.6f}. Average loss: {avg_loss:.6f}')
    return avg_loss, train_acc


def test(model, data_loader, loss_criteria):
    # Switch the model to evaluation mode (so we don't backpropagate)
    model.eval()
    test_loss = 0
    correct = 0
    batch_len = len(data_loader.dataset)
    with torch.no_grad():
        batch_count = 0
        for batch, tensor in enumerate(data_loader):
            batch_count += 1
            data, target = tensor
            # Get the predictions
            out = model(data)

            # calculate the loss
            test_loss += loss_criteria(out, target).item()

            # Calculate the accuracy
            _, predicted = torch.max(out.data, 1)
            correct += torch.sum(abs((out * 13) - (target * 13)) >= 0.5).item()
            
    # Calculate the average loss and total accuracy for this epoch
    avg_loss = test_loss/batch_len
    
    test_acc = 100. * correct / len(data_loader.dataset)
    
    # return average loss for the epoch
    return avg_loss, test_acc


In [32]:
a = torch.Tensor([0.5, 1, 0.6, 0.7])
b = torch.Tensor([0.1, 0.5, 0.1, 0.2])
torch.sum(abs((a ) - (b )) >= 0.1)

tensor(4)

In [33]:
print(next(iter(train_loader)))

[tensor([[4.3478e-01, 1.1458e-01, 5.7143e-01, 0.0000e+00],
        [0.0000e+00, 6.3479e-04, 5.7143e-01, 1.0000e+00],
        [0.0000e+00, 1.3754e-03, 6.4286e-01, 1.0000e+00],
        [0.0000e+00, 6.7711e-03, 5.7143e-01, 0.0000e+00],
        [0.0000e+00, 1.1215e-02, 5.7143e-01, 0.0000e+00],
        [0.0000e+00, 5.2899e-04, 5.7143e-01, 0.0000e+00],
        [0.0000e+00, 2.7402e-02, 7.8571e-01, 0.0000e+00],
        [0.0000e+00, 5.1841e-02, 7.8571e-01, 0.0000e+00],
        [2.1739e-02, 6.7711e-03, 5.7143e-01, 1.0000e+00],
        [0.0000e+00, 8.8870e-03, 5.7143e-01, 0.0000e+00],
        [0.0000e+00, 2.2218e-03, 3.5714e-01, 1.0000e+00],
        [0.0000e+00, 1.0083e-01, 6.4286e-01, 0.0000e+00],
        [2.1739e-02, 8.2522e-03, 6.4286e-01, 0.0000e+00],
        [0.0000e+00, 8.2522e-03, 7.8571e-01, 0.0000e+00],
        [0.0000e+00, 4.4435e-03, 5.7143e-01, 1.0000e+00],
        [0.0000e+00, 4.8667e-03, 6.4286e-01, 0.0000e+00],
        [2.1739e-02, 0.0000e+00, 5.7143e-01, 0.0000e+00],
        [2.17

In [35]:
# Specify the loss criteria (we'll use CrossEntropyLoss for multi-class classification)
# loss_criteria = nn.CrossEntropyLoss()
loss_criteria = nn.MSELoss()
# Use an "Adam" optimizer to adjust weights
# (see https://pytorch.org/docs/stable/optim.html#algorithms for details of supported algorithms)
learning_rate = 5e-3
net = ShauryaNet()
optimizer = torch.optim.Adam(net.parameters(), lr=learning_rate)
optimizer.zero_grad()

# We'll track metrics for each epoch in these arrays
epoch_nums = []
training_loss = []
validation_loss = []

# Train over 50 epochs
epochs = 50
# pbar = tqdm()

for epoch in range(1, epochs + 1):

    # print the epoch number
    
    # Feed training data into the model to optimize the weights
    train_loss, train_acc = train(net, train_loader, optimizer, loss_criteria)
    
    # Feed the test data into the model to check its performance
    test_loss, test_acc = test(net, test_loader, loss_criteria)
    
    # Log the metrics for this epoch
    epoch_nums.append(epoch)
    training_loss.append(train_loss)
    validation_loss.append(test_loss)
    s = f'Epoch: {epoch}. Train loss: {train_loss:0.6f}, Train acc: {train_acc:.6f}, Test loss: {test_loss:0.6f}, Test acc: {test_acc:0.6f}'
    print(s)
    

C:\Users\HP\anaconda3\lib\site-packages\torch\nn\modules\loss.py:530: UserWarning: Using a target size (torch.Size([20])) that is different to the input size (torch.Size([20, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
C:\Users\HP\anaconda3\lib\site-packages\torch\nn\modules\loss.py:530: UserWarning: Using a target size (torch.Size([9])) that is different to the input size (torch.Size([9, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
C:\Users\HP\anaconda3\lib\site-packages\torch\nn\modules\loss.py:530: UserWarning: Using a target size (torch.Size([10])) that is different to the input size (torch.Size([10, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, t

Epoch: 1. Train loss: 0.028581, Train acc: 1701.342282, Test loss: 0.001489, Test acc: 1394.697987
Epoch: 2. Train loss: 0.028197, Train acc: 1726.577181, Test loss: 0.001485, Test acc: 1388.053691
Epoch: 3. Train loss: 0.028068, Train acc: 1743.870246, Test loss: 0.001476, Test acc: 1582.751678
Epoch: 4. Train loss: 0.027954, Train acc: 1769.977629, Test loss: 0.001471, Test acc: 1579.328859
Epoch: 5. Train loss: 0.027895, Train acc: 1777.248322, Test loss: 0.001467, Test acc: 1632.281879
Epoch: 6. Train loss: 0.027852, Train acc: 1782.885906, Test loss: 0.001464, Test acc: 1660.335570
Epoch: 7. Train loss: 0.027822, Train acc: 1784.563758, Test loss: 0.001463, Test acc: 1756.711409
Epoch: 8. Train loss: 0.027799, Train acc: 1786.532438, Test loss: 0.001461, Test acc: 1760.805369
Epoch: 9. Train loss: 0.027783, Train acc: 1786.420582, Test loss: 0.001460, Test acc: 1805.033557
Epoch: 10. Train loss: 0.027767, Train acc: 1787.472036, Test loss: 0.001459, Test acc: 1805.704698
Epoch: 11

### 

In [ ]:
print('hello')